In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bystro.api import auth, annotation
from bystro.proteomics.annotation_interface import get_annotation_result_from_query, execute_query
import pandas as pd
import json

In [3]:
user = auth.login('akotlar@bu.edu', 'gonxug-xubram-nUbny6', 'https://bystro-dev.emory.edu', print_result=False)
index = '661be6b20f228023b55a6927_657a50d4b2d0278938ba791d'

Existing session found, logging out


In [4]:
ancestry_json = None

with open('ancestry_results.json', 'r') as f:
    ancestry_json = json.load(f)

In [5]:
results = []
for row in ancestry_json['results']:
    # print(row)
    max_superpop_prob = 0
    max_superpop_label = None

    for superpop, value in row['superpops'].items():
        mean = (value['lowerBound'] + value['upperBound']) / 2

        if mean > max_superpop_prob:
            max_superpop_label = superpop
            max_superpop_prob = mean
    
    results.append({
       'sample_id': row['sampleId'],
       'superpop_top_hit': superpop,
       'superpop_top_hit_prob': max_superpop_prob
    })

In [6]:
ancestry_df = pd.DataFrame(results)
ancestry_df

,sample_id,superpop_top_hit,superpop_top_hit_prob
0,1805,SAS,0.479617
1,1847,SAS,0.452833
2,4805,SAS,0.494552


In [7]:
results = annotation.get_jobs('completed')
job_id = None
index_name = None
for res in results:
    if 'big_daly' in res.name:
        job_id = res._id
        break
job = annotation.get_jobs(job_id=job_id)

In [8]:
job['outputFileNames']

{'annotation': 'big_daly_vcf-20240414142242561-20240414142242572.annotation.tsv.gz',
 'statistics': {'json': 'big_daly_vcf-20240414142242561-20240414142242572.statistics.json',
  'qc': 'big_daly_vcf-20240414142242561-20240414142242572.statistics.qc.tsv',
  'tab': 'big_daly_vcf-20240414142242561-20240414142242572.statistics.tsv'},
 'config': 'hg38.yml',
 'sampleList': 'big_daly_vcf-20240414142242561-20240414142242572.sample_list',
 'dosageMatrixOutPath': 'big_daly_vcf-20240414142242561-20240414142242572.dosage.feather',
 'header': 'big_daly_vcf-20240414142242561-20240414142242572.annotation.header.json',
 'log': 'big_daly_vcf-20240414142242561-20240414142242572.annotation.log.txt'}

In [9]:
index_name = job['search']['indexName']

In [10]:
job_id

'661be6b20f228023b55a6927'

In [11]:
import pandas as pd
df = pd.read_feather('0.feather')
df

,chrom,pos,vcfPos,inputRef,alt,type,id,locus,sample,dosage,refSeq.name2,refSeq.name,refSeq.exonicAlleleFunction,refSeq.siteType
0,chr1,61862742,61862742,A,C,SNP,.,chr1:61862742:A:C,SKAAWG0403,1,PATJ,NM_001350145,None,intronic
1,chr1,61862742,61862742,A,C,SNP,.,chr1:61862742:A:C,SKAAWG0403,1,PATJ,NM_176877,None,intronic
2,chr1,62099353,62099353,T,C,MULTIALLELIC,.,chr1:62099353:T:C,AA25009,1,PATJ,NM_001350145,None,intronic
3,chr1,62099353,62099353,T,C,MULTIALLELIC,.,chr1:62099353:T:C,AA25009,1,PATJ,NM_176877,None,intronic
4,chr1,62099353,62099353,T,C,MULTIALLELIC,.,chr1:62099353:T:C,11328,-1,PATJ,NM_001350145,None,intronic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6625026,chrX,155127657,155127657,A,G,SNP,rs116550561,chrX:155127657:A:G,WGSB4966,2,None,None,None,None
6625027,chrX,155258730,155258730,A,G,SNP,.,chrX:155258730:A:G,WGSB3658,1,RAB39B,NM_171998,None,UTR3
6625028,chrX,155631867,155631867,A,G,SNP,.,chrX:155631867:A:G,11607,1,SPRY3,NM_001304990,None,intronic
6625029,chrX,155787161,155787161,C,T,SNP,.,chrX:155787161:C:T,AB36282268,1,None,None,None,None


In [ ]:
df.memory_usage(deep=True).sum()

In [ ]:
import pyarrow as pa
import pyarrow.feather as feather

def concatenate_feather_files_memory_mapped(file_list, output_file=None):
    # Open the Feather files using memory mapping
    tables = [feather.read_table(file, memory_map=True) for file in file_list]

    # Concatenate the Arrow Tables
    concatenated_table = pa.concat_tables(tables)
    
    # Optionally, write the concatenated table to a new Feather file
    if output_file:
        feather.write_feather(concatenated_table, output_file)
    
    return concatenated_table


In [ ]:
df = feather.read_feather('concat.feather', memory_map=True)

In [ ]:
df.shape[0]/1e6

In [ ]:
import psutil
import os
psutil.Process(os.getpid()).memory_info().rss / 1e9

In [ ]:
! du -ah 'concat.feather'

In [ ]:
! du -ah '0.feather'

In [ ]:
concatenate_feather_files_memory_mapped(['0.feather', '0.feather', '0.feather', '0.feather', '0.feather', 
                                         '0.feather', '0.feather', '0.feather', '0.feather', '0.feather', '0.feather', '0.feather'], output_file='concat.feather')

In [ ]:
4358583520/1e6

In [ ]:
pwd

In [ ]:
# Default behavior is to show 1 sample per row, e.g. `melt_samples=True`
query_result_df_array_of_structs_select_fields = get_annotation_result_from_query(
    query_string="cadd:>20",
    index_name=index_name,
    bystro_api_auth=user,
    fields=['refSeq.name2', 'refSeq.name', 'refSeq.exonicAlleleFunction', 'refSeq.siteType'],
    explode_field='refSeq.name2',
    tmp_dir='/mnt/ssd1/bystro/python/python/bystro/examples/tmp',
    output_path='foo'
)
query_result_df_array_of_structs_select_fields.head(n=10)

In [ ]:
query_result_df_array_of_structs_select_fields['sample'].apply(lambda x: "FOO" if x == 1847 else "BAR")

In [ ]:
query_result_df_array_of_structs_select_fields.to_csv('query_result_df_array_of_structs_select_fields_exploded.csv')

In [ ]:
query_result_df_array_of_structs_select_fields[query_result_df_array_of_structs_select_fields['locus'] == "chr17:40699291:G:A"]

In [ ]:
merged_df = query_result_df_array_of_structs_select_fields.merge(ancestry_df, left_on='sample', right_on='sample_id')

In [ ]:
# You can choose to also "melt" the dataframe by a field,
# so that when the field has an array value, it is flattened
# When the primary key of the track isn't found, by default we force that 
# field to have 1 value per row, and warn you of this behavior.
# To change it, set `force_flatten_exploded_field=False`
query_result_df = get_annotation_result_from_query(
    query_string="(type:del && (alt:\-6 || alt:\-1)) || (cadd:>40)",
    index_name=index,
    bystro_api_auth=user,
    melt_samples=True,
    explode_field='refSeq.name2',
    fields=['refSeq.name2', 'refSeq.spID', 'refSeq.name'],
    force_flatten_exploded_field=False
)
query_result_df[['refSeq.name2', 'refSeq.spID']]

In [ ]:
# You can disable sample melting by setting `melt_samples=False`
query_result_df = await async_get_annotation_result_from_query(
    query_string="(type:del && (alt:\-6 || alt:\-1)) || (cadd:>40)",
    index_name=index,
    bystro_api_auth=user,
    fields=['refSeq.name2', 'refSeq.name'],
    melt_samples=False,
    explode_field='refSeq.name2'
)
query_result_df.head(n=10)

In [ ]:
# You can also represent tracks that have multiple fields by dictionaries, instead of flattening
# the dictionaries into separate columns, by setting `structs_of_arrays=False`
# This is currently incompatible with `explode_field` and `fields`
query_result_df_array_of_structs_no_melt = get_annotation_result_from_query(
    query_string="*",
    index_name=index,
    bystro_api_auth=user,
    structs_of_arrays=False
)
query_result_df_array_of_structs_no_melt.head(n=10)